In [1]:
import pymcx as mcx
import json
import random
import SimpleITK as sitk
import numpy as np
import jdata


mcxbin="../../MCXStudiov2020/MCXSuite/mcx/bin/mcx"

In [2]:
def better_write_vol_bin(vol, filename, used_type=np.byte):
    f = open(filename, 'wb')
    dmedium = np.array(vol,used_type)
    f.write(dmedium)
    f.close()

In [58]:
img = sitk.ReadImage("../down128.mhd")

dimension = list(img.GetSize())
x = dimension[0]
y = dimension[1]
z = dimension[2]

vol = sitk.GetArrayFromImage(img)

vol = np.where(vol == 0, 1, vol)

vol[:,:,0] = 0
vol[:,0,:] = 0
vol[0,:,:] = 0
vol[:,:,x-1] = 0
vol[:,y-1,:] = 0
vol[z-1,:,:] = 0

vol = vol.astype(np.byte)

binfile = "down128.bin"
better_write_vol_bin(vol, binfile)

In [69]:
vol.shape[2]

128

In [66]:

cfg = mcx.create() #create a default config dictionary

cfg["Domain"]["VolumeFile"] = binfile
cfg["Domain"]["MediaFormat"] = "byte"
cfg["Domain"]["Dim"] = dimension
cfg["Domain"]["OriginType"] = 1
cfg["Domain"]["LengthUnit"] = (1024 / x) * 0.02
cfg["Domain"]["Media"] = [{'mua': 0.0, 'mus': 0.0, 'g': 1.0, 'n': 1.0},
                          {'mua': 0.0, 'mus': 0.0, 'g': 1.0, 'n': 1.0},
                         {'mua': 0.1, 'mus':2.867 , 'g': 0.99, 'n': 1.63},
                         {'mua': 0.35, 'mus': 22.193, 'g': 0.83, 'n': 1.49},
                         {'mua':  2.8, 'mus': 0.0, 'g': 1.0, 'n': 1.333}]


cfg["Forward"]["Dt"] = 5e-09
cfg["Forward"]["T0"] = 0.0
cfg["Forward"]["T1"] = 5e-09


cfg["Session"]["Photons"] = 1e7
cfg["Session"]["RNGSeed"] = random.randint(0,999999999)
cfg["Session"]["ID"] = "cfg_from_python"
cfg["Session"]["DoAutoThread"] = 1


cfg["Optode"]["Source"]["Dir"] =  [0.0, 1, 0]
cfg["Optode"]["Source"]["Param1"] = [0.0, 0.0, 0.0, 0.0]
cfg["Optode"]["Source"]["Param2"] = [0.0, 0.0, 0.0, 0.0]
cfg["Optode"]["Source"]["Pos"] = [x/2, 1,z*3/4]
cfg["Optode"]["Source"]["Type"] = "pencil"

cfg["Optode"]["Detector"] = [{"Pos": [x/2,  y/2,  0.0], "R": 5.0}]


#del cfg["Optode"]["Detector"]
del cfg["Shapes"]


flags = ""
#flags += "-a 1 "
#flags += "--outputformat jnii " # tx3 jnii
flags += "--saveexit 1 "
flags += "--saveref 1 "
flags += "--savedetflag 127 "
#flags += "--bc aaaaaa111111 "
#flags += "--dumpjson"

data = mcx.run(cfg, flag=flags, mcxbin=mcxbin)

###############################################################################
#                      Monte Carlo eXtreme (MCX) -- CUDA                      #
#          Copyright (c) 2009-2020 Qianqian Fang <q.fang at neu.edu>          #
#                             http://mcx.space/                               #
#                                                                             #
# Computational Optics & Translational Imaging (COTI) Lab- http://fanglab.org #
#   Department of Bioengineering, Northeastern University, Boston, MA, USA    #
###############################################################################
#    The MCX Project is funded by the NIH/NIGMS under grant R01-GM114365      #
###############################################################################
$Rev::0313d4$ v2020 $Date::2020-09-06 23:25:03 -04$ by $Author::Qianqian Fang $
###############################################################################
- variant name: [Fermi] compiled by nvcc

In [53]:
jnii = jdata.load("cfg_from_python.jnii")


In [54]:
jdat = jdata.load("cfg_from_python_detp.jdat")

In [55]:
print(jdat["MCXData"]["PhotonData"]["detid"].shape)
print(jdat["MCXData"]["PhotonData"]["nscat"].shape)
print(jdat["MCXData"]["PhotonData"]["ppath"].shape)
print(jdat["MCXData"]["PhotonData"]["mom"].shape)
print(jdat["MCXData"]["PhotonData"]["p"].shape)
print(jdat["MCXData"]["PhotonData"]["v"].shape)
print(jdat["MCXData"]["PhotonData"]["w0"].shape)

#jdat["MCXData"]["PhotonData"]

(28365, 1)
(28365, 4)
(28365, 4)
(28365, 4)
(28365, 3)
(28365, 3)
(28365, 1)


In [56]:
all_layers = jnii["NIFTIData"]

img = sitk.GetImageFromArray(all_layers)
sitk.WriteImage(img, "blub.mhd")

In [63]:
print(data[1].min())
print(data[1].max())

-450515584.0
6652624896.0


In [64]:
fluence_and_dref = data[1]
dref = np.where(fluence_and_dref < 0, -fluence_and_dref, 0)
dref = np.where(dref == 0, 10**-20, dref)
dref = np.log(dref)
dref = sitk.GetImageFromArray(dref)
sitk.WriteImage(dref, "dref.mhd")

In [442]:
for i in range(10):
    layer = all_layers[:,:,:,i]
    print(layer.flatten().max())
    if layer.max() > 0.0:
        img = sitk.GetImageFromArray(layer)
        sitk.WriteImage(img, "blub"+str(i)+".mhd")

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [443]:
np.array_equal(all_layers[:,:,:,9], all_layers[:,:,:,5])

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [401]:
 all_layers[:,:,:,2].flatten().max()

1.6373824e-07

In [452]:
data

((array([[-1.        ,  0.        ,  2.        , ..., -0.18568276,
           0.91388571,  1.        ],
         [-1.        ,  0.        ,  4.        , ...,  0.03371319,
           0.99435586,  1.        ],
         [-1.        ,  0.        ,  3.        , ..., -0.30029836,
           0.94502717,  1.        ],
         ...,
         [-1.        ,  0.        ,  6.        , ..., -0.90127796,
          -0.34897351,  1.        ],
         [-1.        ,  0.        , 11.        , ..., -0.48507562,
           0.52853417,  1.        ],
         [-1.        ,  0.        ,  5.        , ..., -0.20103237,
           0.89341414,  1.        ]]),
  {'version': 1,
   'medianum': 4,
   'detnum': 1,
   'recordnum': 20,
   'totalphoton': 10000000,
   'detectedphoton': 1000000,
   'savedphoton': 1000000,
   'lengthunit': 0.1599999964237213,
   'seedbyte': 0,
   'normalizer': 781.25,
   'respin': 1,
   'srcnum': 1,
   'savedetflag': 127},
  []),
 array([[[[0.],
          [0.],
          [0.],
          ...

In [ ]:
mcx.load_mc2()

In [460]:
data[0][1]

{'version': 1,
 'medianum': 4,
 'detnum': 1,
 'recordnum': 20,
 'totalphoton': 10000000,
 'detectedphoton': 1000000,
 'savedphoton': 1000000,
 'lengthunit': 0.1599999964237213,
 'seedbyte': 0,
 'normalizer': 781.25,
 'respin': 1,
 'srcnum': 1,
 'savedetflag': 127}

In [464]:
data[0][0].shape

(1000000, 20)

In [465]:
data[1].shape

(128, 128, 65, 1)